In [84]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
from scipy import stats

In [85]:
Y = np.arange (10)
X = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
n = 100

In [86]:
#подсчёт Коломогорова
X_sample = []
for i in range(len(X)):
    for j in range(X[i]):
        X_sample.append(i)

In [87]:
F_acc = stats.uniform.cdf(Y, loc=-0.5, scale=10)
F_emp = []

In [88]:
for i in Y:
    F_emp.append(np.sum(X_sample < i) / n)

delta = (n**0.5) * max([abs(F_acc[i] - F_emp[i]) for i in range(len(Y))])
print("delta =",delta)

delta = 1.5999999999999996


In [89]:
#Подсчет Хи2, Колмогорова
limits = [-np.inf]
for i in range(len(Y) - 1):
    limits.append(i + 0.5)
limits.append(np.inf)

print(limits)

[-inf, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, inf]


In [90]:
def normal_pdf(x, a, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-(x - a)**2 / (2 * sigma**2))

In [91]:
a, sigma = np.linspace(2, 8, 100), np.linspace(2, 8, 100)
L_max, a_norm, sigma_norm  = 0, 0, 0

In [92]:
for sigma_i in sigma:
    for a_i in a:
        L_i = 1

        for i in range(len(X)):
            p_i = integrate.quad(normal_pdf, limits[i], limits[i + 1], args = (a_i, sigma_i))[0]
            L_i *= p_i ** X[i]

        if L_i > L_max:
            L_max = L_i
            sigma_norm = sigma_i
            a_norm = a_i

In [93]:
print("a =", a_norm, "\nsigma =", sigma_norm)

a = 4.787878787878788 
sigma = 2.666666666666667


In [94]:
delta = 0
for i in range(len(X)):
    p_i = integrate.quad(normal_pdf, limits[i], limits[i + 1], args = (a_norm, sigma_norm))[0]
    delta += (X[i] - n * p_i)**2 / (n * p_i)
print("delta =", delta)

delta = 9.819409868080811


In [95]:
N = 50000
delta_K = []

In [96]:
for i in range(N):
    sample_i = np.random.normal(a_norm, sigma_norm, n)
    tmp = np.linspace(0, max(max(sample_i), 9), 100)

    F_emp_star = [np.sum(sample_i < j) / n for j in tmp]
    F_emp = [np.sum(X_sample < j) / n for j in tmp]

    delta_i = (n ** 0.5) * max([abs(F_emp_star[i] - F_emp[i]) for i in range(len(tmp))])
    delta_K.append(delta_i)

In [97]:
delta_wave = (n ** 0.5) * np.max (abs([np.sum(X_sample < j) / n for j in Y] - stats.norm(a_norm, sigma_norm).cdf(Y)))

delta_K_var = np.sort(delta_K)

k = len(delta_K_var [delta_K_var < delta_wave])
p_value = 1 - ((k - 1)/N)

print("delta~ =", delta_wave)
print("p-value =", p_value)

delta~ = 0.8581058774325867
p-value = 0.9939


In [98]:
#Хи2 метод моментов
a_mm = np.mean(X_sample)
sigma_mm = np.std(X_sample)

print(a_mm, sigma_mm)

4.77 2.505414137423193


In [99]:
delta_mm = []
N = 10000

In [100]:
for i in range (N):
    sample_i = np.random.normal(a_mm, sigma_mm, n)
    mean_i = np.mean(sample_i)
    sigma_i = np.std(sample_i)

    p_i, m_i = [], []

    for j in range(len(X)):
        p_i.append(integrate.quad(normal_pdf, limits[j], limits[j + 1], args = (mean_i, sigma_i))[0])
        m_i.append(len(sample_i [(sample_i >= limits[j]) & (sample_i <= limits[j + 1])]))
    delta_mm.append(sum(((m_i[i] - n * p_i[i]) ** 2) / (n * (p_i[i])) for i in range(len(X))))

In [101]:
delta_mm_var = np.sort(delta_mm)
p_value = 1 - ((len(delta_mm_var[delta_mm_var < delta]) - 1) / N)
print("p-value =", p_value, "\nНет основания отвергать гипотезу")

p-value = 0.20879999999999999 
Нет основания отвергать гипотезу
